In [29]:
import subprocess, shutil, datetime
from pathlib import Path

import numpy as np
import pandas as pd

from vtu import PyVtu
from ts_auto_wrapper import TSWrapper

In [2]:
ts=TSWrapper(Path("/opt/workspace/msc_project/cluster-trisurf"))
# ts=TSWrapper(Path("/opt/workspace/msc_project/trisurf_samo/trisurf-ng/"))
trisurf_test = Path('.')/"trisurf_test"

In [3]:
test_folder=None
tape=None
extra_args = []
destroy_timesteps_at_end=True # destroy all timesteps after the test
timeout = 750 # seconds to timeout for each `trisurf` test, fit for samo trisurf with nshell=17

In [4]:
test_folder = trisurf_test/'2023_05_29_main_egg_carton'
# test_folder = trisurf_test/'2023_05_29_anisotropy_samo_version'
extra_args = ['-c adhesion_model=1,adhesion_geometry=6,adhesion_scale=2']
destroy_timesteps_at_end=False # destroy all timesteps after the test
timeout = 120 # seconds to timeout for each `trisurf` test
# timeout = 750

### Make a new folder if there isn't one ###

In [5]:
new_folder=None
if not test_folder:
    # make up some appropriate folder
    now = datetime.datetime.now()
    ret = subprocess.run(['git', 'status'],cwd=ts.path_to_trisurf,capture_output=True)
    branch = ret.stdout.decode().partition('On branch ')[-1].splitlines()[0].strip()
    new_folder = f'{now.year:04}_{now.month:02}_{now.day:02}_{branch}'
    test_folder = trisurf_test/new_folder
new_folder

In [6]:
if not test_folder.exists():
    test_folder.mkdir()
    test_folder.joinpath('test_results').mkdir()
elif new_folder is not None:
    raise ValueError(f'folder {test_folder} for today already exists: Rename or delete to run test')

### copy the tape to the new folder ###

In [7]:
if tape is None:
    tape=ts.path_to_trisurf/"src/tape"
shutil.copy(tape,test_folder/"tape")

PosixPath('trisurf_test/2023_05_29_main_egg_carton/tape')

## Check force from tape ##

In [8]:
ret = subprocess.run([ts.path_to_trisurf/"src/trisurf", "--force-from-tape", *extra_args],
                     cwd=test_folder, timeout=timeout, capture_output=True)

In [9]:
with open(test_folder/"test_results"/"force_stdout.txt",'w') as f: 
    f.write(ret.stdout.decode()+'\n'+ret.stderr.decode())
shutil.copy(test_folder/"statistics.csv",test_folder/"test_results"/"force_statistics.csv")
print(ret.stdout.decode())

[2023-05-29 15:28:39] TRISURF-NG v. 24764b9, compiled on: May 29 2023 15:25:02.
[2023-05-29 15:28:39] Programming done by: Samo Penic and Miha Fosnaric
[2023-05-29 15:28:39] Released under terms of GPLv3
[2023-05-29 15:28:39] Starting program...

[2023-05-29 15:28:39] ************************************************
[2023-05-29 15:28:39] **** Generating initial geometry from tape *****
[2023-05-29 15:28:39] ************************************************

[2023-05-29 15:28:39] Starting initial_distribution on vesicle with 10 shells!...
[2023-05-29 15:28:39] initial_distribution finished!
[2023-05-29 15:28:39] using debug curvature model 15 (use new energy): set to 7 to use the old energy method
[2023-05-29 15:28:39] simulation seed 1685363319
[2023-05-29 15:28:39] Setting volume V0=1041.99378875998718286
[2023-05-29 15:28:39] Setting area A0=623.53829072480630202
[2023-05-29 15:28:41] Done 1 out of 10 iterations (x 1000 MC sweeps).
[2023-05-29 15:28:43] Done 2 out of 10 iterations (x 

## Check start from binary dump ##

In [10]:
ret = subprocess.run([ts.path_to_trisurf/"src/trisurf","--reset-iteration-count", *extra_args],
                     cwd=test_folder, timeout=timeout, capture_output=True)

In [11]:
with open(test_folder/"test_results"/"reset_stdout.txt",'w') as f: 
    f.write(ret.stdout.decode()+'\n'+ret.stderr.decode())
shutil.copy(test_folder/"statistics.csv",test_folder/"test_results"/"reset_statistics.csv")
print(ret.stdout.decode())

[2023-05-29 15:29:05] TRISURF-NG v. 24764b9, compiled on: May 29 2023 15:25:02.
[2023-05-29 15:29:05] Programming done by: Samo Penic and Miha Fosnaric
[2023-05-29 15:29:05] Released under terms of GPLv3
[2023-05-29 15:29:05] Starting program...

[2023-05-29 15:29:05] **********************************************************************
[2023-05-29 15:29:05] **** Recreating vesicle from dump file and continuing simulation *****
[2023-05-29 15:29:05] **********************************************************************

[2023-05-29 15:29:05] using debug curvature model 15 (use new energy): set to 7 to use the old energy method
[2023-05-29 15:29:05] simulation seed 1685363345
[2023-05-29 15:29:05] Setting volume V0=1619.40294767925843189
[2023-05-29 15:29:05] Setting area A0=758.95771961849698073
[2023-05-29 15:29:07] Done 1 out of 10 iterations (x 1000 MC sweeps).
[2023-05-29 15:29:10] Done 2 out of 10 iterations (x 1000 MC sweeps).
[2023-05-29 15:29:13] Done 3 out of 10 iterations (x

## Checking start from vtu ##

In [12]:
test_folder.joinpath('.status').unlink() # remove status so it resets iterations

In [13]:
ret = subprocess.run([ts.path_to_trisurf/"src/trisurf",
                      "--restore-from-vtk",'timestep_000000.vtu', *extra_args],
                     cwd=test_folder, timeout=timeout, capture_output=True)

In [14]:
with open(test_folder/"test_results"/"restore_stdout.txt",'w') as f: 
    f.write(ret.stdout.decode()+'\n'+ret.stderr.decode())
shutil.copy(test_folder/"statistics.csv",test_folder/"test_results"/"restore_statistics.csv")
print(ret.stdout.decode())

[2023-05-29 15:29:32] TRISURF-NG v. 24764b9, compiled on: May 29 2023 15:25:02.
[2023-05-29 15:29:32] Programming done by: Samo Penic and Miha Fosnaric
[2023-05-29 15:29:32] Released under terms of GPLv3
[2023-05-29 15:29:32] Starting program...

[2023-05-29 15:29:32] ************************************************
[2023-05-29 15:29:32] **** Restoring vesicle from VTK points list ****
[2023-05-29 15:29:32] ************************************************

[2023-05-29 15:29:32] No .status file. The iteration count will start from 0
[2023-05-29 15:29:32] using debug curvature model 15 (use new energy): set to 7 to use the old energy method
[2023-05-29 15:29:32] simulation seed 1685363372
[2023-05-29 15:29:32] Setting volume V0=1625.83364468069930808
[2023-05-29 15:29:32] Setting area A0=760.69137689373678768
[2023-05-29 15:29:35] Done 1 out of 10 iterations (x 1000 MC sweeps).
[2023-05-29 15:29:37] Done 2 out of 10 iterations (x 1000 MC sweeps).
[2023-05-29 15:29:39] Done 3 out of 10 it

Files are cleaned up at the end!

## More tests ##

### so far: load 1st file as both PyVtu and vesicle. See what is the difference

In [15]:
try:
    fileloc = list(sorted(test_folder.glob('timestep*')))[-1]
except NameError:
    test_folder = list(sorted(trisurf_test.glob('./[!.]*')))[-1]
    fileloc = list(sorted(test_folder.glob('timestep*')))[-1]
fileloc

PosixPath('trisurf_test/2023_05_29_main_egg_carton/timestep_000000.vtu')

In [16]:
vesicle = ts.parseDump(fileloc)

In [17]:
v = PyVtu(fileloc)
v2 = PyVtu(fileloc) # we will place here the values from vesicle

In [18]:
def iter_xlist(vesiclePtr,xlist_name='vlist'):
    """Iterate over some vesicle->xlist->xs. Default vesicle->vlist->vtx[:vlist.n]."""
    lst = vesiclePtr.contents.__getattribute__(xlist_name)
    if xlist_name=='vlist':
        n, lst = lst.contents.n, lst.contents.vtx
    elif xlist_name=='clist':
        n, lst = lst.contents.cellno, lst.contents.cell
    elif xlist_name=='tlist':
        n, lst = lst.contents.n, lst.contents.tria
    elif xlist_name=='blist':
        n, lst = lst.contents.n, lst.contents.bond
    else:
        raise ValueError(f'{xlist_name} not in vlist,clist,tlist,blist')
    for i in range(n):
        yield lst[i].contents


In [19]:
def replace_pyvtu_values_from_vesicle(v,vesiclePtr):
    """Replace the values the PyVtu arrays with values from the wrapper."""
    blen = vesicle.contents.blist.contents.n
    tlen = vesicle.contents.tlist.contents.n
    v.pos[:] = [(x.x,x.y,x.z) for x in iter_xlist(vesicle)]
    v.add_array('neigh_no',np.array([x.neigh_no for x in iter_xlist(vesicle)]))
    v.c0[:] = [x.c for x in iter_xlist(vesicle)]
    v.e[:] = [x.energy for x in iter_xlist(vesicle)]
    # newer version stuff (can fail but who cares?)
    v.w[:] = [x.w for x in iter_xlist(vesicle)]
    v.f0[:] = [x.f for x in iter_xlist(vesicle)]
    v.face_normal[-tlen:] = np.array([(x.xnorm, x.ynorm, x.znorm) for x in iter_xlist(vesicle,"tlist")])
    v.normal[:] = [(x.nx,x.ny,x.nz) for x in iter_xlist(vesicle)]
    v.force[:] =[(x.fx,x.fy,x.fz) for x in iter_xlist(vesicle)]
    v.director[:] = [(x.dx,x.dy,x.dz) for x in iter_xlist(vesicle)]
    v.eig0[:] = [x.eig0[:] for x in iter_xlist(vesicle)]
    v.eig1[:] = [x.eig1[:] for x in iter_xlist(vesicle)]
    v.eig2[:] = [x.eig2[:] for x in iter_xlist(vesicle)]
    v.ad_w[:] = [x.ad_w for x in iter_xlist(vesicle)]
    v.type[:] = [ts.byte_to_int(x.type) for x in iter_xlist(vesicle)]
    v.eigenvalue_0[:] = [x.eig_v0 for x in iter_xlist(vesicle)]
    v.eigenvalue_1[:] = [x.eig_v1 for x in iter_xlist(vesicle)]
    v.eigenvalue_2[:] = [x.eig_v2 for x in iter_xlist(vesicle)]
    v.k[:] = [x.xk for x in iter_xlist(vesicle)]
    v.k2[:] = [x.xk2 for x in iter_xlist(vesicle)]
    v.mean_curvature[:] = [x.mean_curvature for x in iter_xlist(vesicle)]
    v.gaussian_curvature[:] = [x.gaussian_curvature for x in iter_xlist(vesicle)]
    v.mean_curvature2[:] = [x.mean_curvature2 for x in iter_xlist(vesicle)]
    v.gaussian_curvature2[:] = [x.gaussian_curvature2 for x in iter_xlist(vesicle)]
    v.add_array('circumcenter',np.array(([(0,0,0)]*blen)+[(x.xcirc,x.ycirc,x.zcirc) for x in iter_xlist(vesicle,'tlist')]),parent='CellData')
    

In [20]:
try:
    replace_pyvtu_values_from_vesicle(v2,vesicle)
except AttributeError as e:
    print(f'partial replacement: {e}')

#### Neighbor counts

In [21]:
_, counts = np.unique(v.blist, return_counts=True)
assert (counts==v2.neigh_no).all()

#### Neighbors are ordered

They should be ordered, but the array is circularly shifted i.e. `[0,1,2]==[2,0,1]` but not `[2,1,0]` or `[3,1,2]`

In [22]:
ordered=[]
ordered_identical=[]
for i in v.indices:
    vtx = vesicle.contents.vlist.contents.vtx[i].contents
    neighs = np.array([x.contents.idx for x in vtx.neigh[:vtx.neigh_no]])
    neighs_v = v.get_ordered_neighbors(i)
    ordered_identical.append((neighs==neighs_v).all())
    ordered.append(any([(neighs==np.roll(neighs_v,i)).all() for i, _ in enumerate(neighs) ]))

In [23]:
assert all(ordered)
all(ordered), all(ordered_identical) # should be (True,False)

(True, False)

### Make sure all values are close

In [24]:
close_arrays={k:np.allclose(v._arrays[k],v2._arrays[k]) for k in v._arrays}
close_arrays

{'pos': True,
 'blist': True,
 'tlist': True,
 'indices': True,
 'type': True,
 'c0': True,
 'w': True,
 'f0': True,
 'ad_w': True,
 'd0': True,
 'mean_curvature': True,
 'gaussian_curvature': True,
 'mean_curvature2': True,
 'gaussian_curvature2': True,
 'eigenvalue_0': True,
 'eigenvalue_1': True,
 'eigenvalue_2': True,
 'k': True,
 'k2': True,
 'e': False,
 'mean_energy': True,
 'gaussian_energy': True,
 'mean_energy2': True,
 'gaussian_energy2': True,
 'normal': True,
 'normal_unused_debug': True,
 'force': False,
 'director': False,
 'eig0': False,
 'eig1': False,
 'eig2': True,
 'bonding_energy': True,
 'face_normal': True}

#### check gaussian nonesense

In [27]:
if 'gaussian_curvature' in v._arrays:
    gdiff = v.gaussian_curvature-v.gaussian_curvature2
    print(f'gaussian curvature: max difference: {gdiff.max():.4e},'
          f' avg ± std: {gdiff.mean():.4e} ± {gdiff.std():.4e}')

gaussian curvature: max difference: 9.1898e-02, avg ± std: 5.8667e-03 ± 1.3730e-02


In [25]:
big_diffs = {k:(v._arrays[k]-v2._arrays[k])
             for k,val in close_arrays.items() if not val}
{k: f'max difference: {v.max():.4e}, avg ± std: {v.mean():.4e} ± {v.std():.4e}' for k, v in big_diffs.items()}

{'e': 'max difference: 4.9208e-03, avg ± std: -7.4949e-05 ± 1.0177e-03',
 'force': 'max difference: 4.5151e-02, avg ± std: 1.1332e-04 ± 4.8347e-03',
 'director': 'max difference: 2.8749e-03, avg ± std: 2.8693e-06 ± 2.1649e-04',
 'eig0': 'max difference: 1.5512e+00, avg ± std: 2.2118e-03 ± 5.1489e-02',
 'eig1': 'max difference: 1.4652e+00, avg ± std: 1.1544e-04 ± 5.1537e-02'}

In [40]:
# vertex_dataframe=pd.DataFrame({k:val for k,val in v._arrays.items() if val.shape==v.indices.shape})
# vertex_dataframe.describe()

#### Destory files

In [41]:
if destroy_timesteps_at_end:
    files_to_remove = list(test_folder.glob('timestep*'))
    excess_files = ('dump.bin','dout','output.pvd','statistics.csv')
    files_to_remove.extend(test_folder/x for x in excess_files)
    [file.unlink() for file in files_to_remove if file.exists()]